In [1]:
import pandas as pd
import numpy as np
import transformers
from transformers import (
    pipeline,
    TextClassificationPipeline,
    AutoTokenizer
)
import torch

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0, 1),clip=True)
scaler.fit([[0], [100]])
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)
model_name = 'distilroberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
from peft import get_peft_model, LoraConfig, TaskType, PeftModel

MAX_LEN = 512
# TRAIN_BATCH_SIZE = 32
# VALID_BATCH_SIZE = 32
# EPOCHS = 10
# LEARNING_RATE = 3e-05


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



cuda


In [2]:
data = pd.read_parquet("test_drob_ib_revised.gzip")

In [5]:
model_path = "./drob-ib-revised-output/"
model_c = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
# model_c.config.problem_type = "multi_label_classification"
model_c = PeftModel.from_pretrained(model_c, model_path)
model_c = model_c.merge_and_unload()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
data

,ingr,tags
21783,"0.5 lb. turkey breast, cubed, soy sauce, 0.5 t...","[1, 0]"
102326,"1 c. salad oil, 2 eggs, 2 c. sugar, 2.5 c. sel...","[1, 0]"
431435,"1.2 (12 oz.) pkg. butterscotch chips, 0.4 c. b...","[0, 1]"
292881,"peel of 2 oranges, peel of 2 lemons, peel of 1...","[0, 1]"
385125,"4 c. flour, sifted 4 times, 1 lb. butter, soft...","[0, 1]"
...,...,...
153523,"3 c. cooked rice, 1 can creamed corn, 1 lb. gr...","[1, 0]"
386467,"6.4 c. diced, peeled potatoes, 11.52 c. choppe...","[0, 1]"
295885,"3 qt. peeled and chopped ripe tomatoes, 11.2 l...","[0, 1]"
161548,"1 bag Success rice, 0.75 lb. extra-lean ground...","[1, 0]"


In [7]:
# class logitpipeline(TextClassificationPipeline):
#     def postprocess(self, model_outputs):
#         best_class = model_outputs["logits"]
#         return best_class
pipe = TextClassificationPipeline(model=model_c, tokenizer=tokenizer)

In [7]:
pipe("Cook for 15.0 minutes until softened.")

[{'label': 'LABEL_1', 'score': 1.0}]

In [8]:
ingr = data.ingr.tolist()

In [9]:
def data():
    for i in range(len(ingr)):
        yield ingr[i]

In [10]:
def data():
    for i in range(len(ingr)):
        yield ingr[i]
output = []
count = 0
# for s in sents:
#     count += 1
#     if count % 100 ==0:
#         print(count)
#     output.append(pipe(s))
for out in pipe(data()):
    count += 1
    if count % 100 ==0:
        print(count)
    output.append(out)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [11]:
out_df = pd.DataFrame({"output":output})

In [12]:
out_df.to_parquet("pred_drob_revised.gzip", compression='gzip')

In [13]:
out_df

,output
0,"{'label': 'LABEL_0', 'score': 1.0}"
1,"{'label': 'LABEL_0', 'score': 1.0}"
2,"{'label': 'LABEL_1', 'score': 1.0}"
3,"{'label': 'LABEL_1', 'score': 1.0}"
4,"{'label': 'LABEL_1', 'score': 1.0}"
...,...
49183,"{'label': 'LABEL_0', 'score': 1.0}"
49184,"{'label': 'LABEL_1', 'score': 1.0}"
49185,"{'label': 'LABEL_1', 'score': 1.0}"
49186,"{'label': 'LABEL_0', 'score': 1.0}"


In [25]:
# encodings = tokenizer.batch_encode_plus(
#                     data['sentences'].tolist(),
#                     add_special_tokens=True,
#                     max_length=MAX_LEN,
#                     padding='max_length',
#                     return_token_type_ids=True,
#                     truncation=True,
#                     return_tensors="pt"
#                 )

In [7]:
# # def enc(sent)
# encodings = tokenizer.encode_plus(
#                     data['sentences'].tolist()[0],
#                     add_special_tokens=True,
#                     max_length=MAX_LEN,
#                     padding='max_length',
#                     return_token_type_ids=True,
#                     truncation=True,
#                     return_tensors="pt"
#                 )
# ret = [val.clone().detach().to("cuda") for val in encodings.values()]
# ret

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [5]:
# class cooking_time_model(torch.nn.Module):
#     def __init__(self):
#         super(cooking_time_model, self).__init__()
#         self.l1 = transformers.RobertaModel.from_pretrained(pretrained)
#         self.l2 = torch.nn.Dropout(0.3)
#         self.l3 = torch.nn.Linear(768, MAX_LEN)

#     def forward(self, ids, mask, token_type_ids):
#         output_1 = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
#         output_2 = self.l2(output_1.pooler_output)
#         output = self.l3(output_2)
#         return output
model_path = "./drob-mfst-nolora-output/checkpoint-17500"
model_c = transformers.AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=1, load_in_8bit = True)
model_c.config.problem_type = "regression"
# model_c = cooking_time_model()
# model_c.to(device)
# def count_parameters(modelr):
#     return sum(p.numel() for p in modelr.parameters() if p.requires_grad)

# print(f'The model has {count_parameters(model_c):,} trainable parameters')

In [38]:
len(encodings)

3

In [6]:
model_c.eval()
with torch.no_grad():
    output = model_c(input_ids=ret[0],attention_mask=ret[1],token_type_ids=ret[2])

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [22]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min

In [23]:
model_best, _, _, l1 = load_ckp("best_model_a3.pt", model_c, optimizer)
model_current, _, _, l2 = load_ckp("current_checkpoint_a3.pt", model_c, optimizer)
print(l1==l2)
print(l1, l2)

True
0.00022455735384061483 0.00022455735384061483


In [11]:
def submission(dataloader, model_selected): # Code for submission
    preds = []
    targets = []
    model_selected.eval()
    with torch.no_grad():
        for batch_idx, (id, mask, tokentype, target) in enumerate(dataloader):
            outputs = model_selected(id, mask, tokentype)
            preds.append(list(outputs.logits))
            targets.append(list(target))
            if (batch_idx+1)%50 == 0:
                print("~"*15,f"{batch_idx+1}/{len(dataloader)}({(batch_idx+1)/len(dataloader)})","~"*15, end="\r")
            
    return preds, targets
ret, tar = submission(test_loader, model_best)

In [12]:
ltar = [l[0].tolist() for l in tar]
lret = [l[0].tolist()[0] for l in ret]
ltar = [round(l, 4) for l in ltar]
lret = [round(l, 4) if l>=0 else 0 for l in lret]
# lret = [l[0] for l in ret]

In [13]:
# for p, r in zip(ltar, lret):
#     # print(p, r)
#     print(abs(p-r)*100)
err = [abs(p-r) for p,r in zip(ltar,lret)]
print(np.mean(err))

0.006316959055973027


In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0, 1),clip=True)
scaler.fit([[0], [100]])
err_min = scaler.inverse_transform([[i] for i in err])
# for m, e in zip(err_min, err):
#     print(m[0], e)
print("error stats:")
print("average:", np.mean(err_min))
print("max:", *err_min[np.argmax(err_min)])
print("max error target(minutes)",(scaler.inverse_transform([[data.iloc[np.argmax(err_min)][1]]])[0][0]))
print("max error prediction(minutes)",scaler.inverse_transform([[lret[np.argmax(err_min)]]])[0][0])
print("max error recipe:")
print(data.iloc[np.argmax(err_min)][0])
# data


error stats:
average: 0.6316959055973027
max: 92.7
max error target(minutes) 93.5
max error prediction(minutes) 0.8
max error recipe:
1 elephant, 2 rabbits (optional), 2 pails salt, 1 (8 oz.) canister black pepper, 3 bushels onions, <mask> gal. water, 6 pails flour


In [40]:
# test_input = ["<mask> lb. lean beef, cut across the grain into thin slices (you need a quick-cooking cut, such as minute steak), 1 lemongrass stalk, trimmed and finely chopped, <mask> tbsp. soy sauce, 2 tbs. fish sauce, 4 tsp. brown sugar, 0.5 tsp. chilli flakes, 1 lime, juiced, 3 tbsp. vegetable oil, 1 green pepper, thinly sliced, 2 bunches of spring onions, green and white parts separated and finely sliced, 6 garlic cloves, finely chopped, 1 tbsp. grated ginger, small bunch of basil, or purple basil, leaves picked and roughly chopped, cooked rice (about 250g uncooked weight), or cooked rice noodles, 50g roasted peanuts, roughly chopped"]
test_input = ['6 slices bacon, chopped, 0.5 c. chopped onion, <mask> c. chopped celery, 0.5 c. chopped green bell pepper, 3 cloves garlic, minced, 0.5 tsp. kosher salt, 0.5 tsp. freshly ground black pepper, 0.5 tsp. dried thyme, 0.25 tsp. cayenne pepper, 2 (12 ounce) packages frozen black-eyed peas, 4 c. reduced-sodium chicken broth, 2 c. cooked long grain rice, 0.25 c. chopped green onions, or as needed']
# test_input = ['<mask> lb. lean beef, cut across the grain into thin slices (you need a quick-cooking cut, such as minute steak), 1 lemongrass stalk, trimmed and finely chopped, <mask> tbsp. soy sauce, 2 tbsp. fish sauce, 4 tsp. brown sugar, 0.5 tsp. chili flakes, 1 lime, juiced, 3 tbsp. vegetable oil, 1 green pepper, thinly sliced, 2 bunches of spring onions, green and white parts separated and finely sliced, 6 garlic cloves, finely chopped, 1 tbsp. grated ginger, small bunch of basil, or purple basil, leaves picked and roughly chopped, cooked rice (about 250g uncooked weight), or cooked rice noodles, 50g roasted peanuts, roughly chopped']
test_input = ["0.25 c. unsalted butter, 0.25 c. all-purpose flour, 1 medium onion, chopped, 1 c. chopped celery, 0.67 c. chopped red bell pepper, 0.67 c. chopped green bell pepper, 2 cloves garlic, minced, 2 tsp. Cajun seasoning, 0.5 c. chicken broth, 0.5 lb. thinly sliced, fully cooked smoked sausage, 0.25 tsp. crushed red pepper flakes, 0.5 tsp. freshly ground black pepper, salt to taste (see Cook's Notes), 0.5 lb. raw shrimp, peeled and deveined (see Cook's Notes), 0.5 c. heavy whipping cream, <mask> oz. spaghetti, cooked, chopped fresh parsley for garnish (optional), lemon slices for garnish (optional)"]
def test_pipeline(test_input):
    single_df = pd.DataFrame({"ingr":test_input, "tags":[0.0]})
    # single_df
    s_test_customset = CustomDataset(single_df, tokenizer, MAX_LEN)
    s_test_loader = DataLoader(s_test_customset, batch_size=1, shuffle=False, collate_fn=colbatch)
    with torch.no_grad():
        for _, (id, mask, tokentype, _) in enumerate(s_test_loader):
            outputs = model_best(id, mask, tokentype)
            print(outputs.logits.tolist()[0][0])
            print(scaler.inverse_transform(outputs.logits.tolist())[0][0])
            return(outputs.logits.tolist()[0][0])
            
            # return outputs.logits.tolist(), scaler.inverse_transform(outputs.logits.tolist())
test_pipeline(test_input)
scaler.transform([[12]])

0.08980672806501389
8.980672806501389


array([[0.12]])